# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1ogme9BJeHb2IZ6UREavUorF--nnxoWCYAAi8AZ4Q5jQ/edit?usp=sharing) and [grading rubric](https://docs.google.com/document/d/1XI9Yq_e-U-D3iH4jTPAtNteeP2Q9mtJ9NKbePWKeN_g/edit?usp=sharing)

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [61]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

In [62]:
!createdb aendta_db

createdb: error: database creation failed: ERROR:  database "aendta_db" already exists


In [63]:
!psql --dbname aendta_db -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [64]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "Lowr54ShQCxjzTIFPwsC6ZSPm"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "aendta_db"
DB_USER = "alexander"
DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [65]:
# Make sure the QUERY_DIRECTORY & DATA_DIR exists
if not DATA_DIR.exists():
    DATA_DIR.mkdir()
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [66]:
import urllib.parse
import requests
import json
from pathlib import Path

DATA_DIR = Path("data")

def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    filename = DATA_DIR / (url_path.replace("/", "_") + ".geojson") 
    
    if force or not filename.exists(): 
        print(f"Downloading {url} to {filename}...")
        response = requests.get(url)
        response.raise_for_status()  
        
        with open(filename, "w") as f:
            json.dump(response.json(), f) 
        
        print(f"Done downloading {url}.")
    
    else:
        print(f"Reading from {filename}...")
    
    return filename

In [67]:
def load_and_clean_zipcodes(zipcode_datafile):
    zipcodes_df = pd.read_csv("data/nyc_zipcodes-1.csv")
    keep_columns1 = ["ZIPCODE", "PO_NAME", "POPULATION", "BLDGZIP", "AREA", "COUNTY", "CTY_FIPS"]
    zipcodes_df = zipcodes_df[keep_columns1]
    zipcodes_df = zipcodes_df.dropna()
    zipcodes_df.columns = zipcodes_df.columns.str.strip().str.lower()
    zipcodes_df["zipcode"] = zipcodes_df["zipcode"].astype(str)
        
    return zipcodes_df
zipcode_datafile = 'data/nyc_zipcodes-1.csv'  # Replace with your file path
df_zipcodes = load_and_clean_zipcodes(zipcode_datafile)
df_zipcodes

,zipcode,po_name,population,bldgzip,area,county,cty_fips
0,11436,Jamaica,18681.0,0,2.269930e+07,Queens,81
1,11213,Brooklyn,62426.0,0,2.963100e+07,Kings,47
2,11212,Brooklyn,83866.0,0,4.197210e+07,Kings,47
3,11225,Brooklyn,56527.0,0,2.369863e+07,Kings,47
4,11218,Brooklyn,72280.0,0,3.686880e+07,Kings,47
...,...,...,...,...,...,...,...
258,10310,Staten Island,25003.0,0,5.346328e+07,Richmond,85
259,11693,Far Rockaway,11052.0,0,3.497516e+06,Kings,47
260,11249,Brooklyn,28481.0,0,1.777221e+07,Kings,47
261,10162,New York,0.0,1,2.103489e+04,New York,61


In [68]:
import requests
import os
from pathlib import Path

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)  
application_token = os.getenv(NYC_DATA_APP_TOKEN)
url_311 = "https://data.cityofnewyork.us/resource/erm2-nwe9.json"

def download_and_clean_311_data(_331_datafile):
    headers = {"X-App-Token": NYC_DATA_APP_TOKEN}
    response = requests.get(url_311, headers=headers)
    response.raise_for_status() 
    
    filename_311 = data_dir / "311_data.json"
    with open(filename_311, "w") as f:
        f.write(response.text) 
    
    data_311 = pd.read_json(filename_311) 
    keep_columns2 = ["created_date", "complaint_type", "latitude", "longitude", "borough"]
    data_311 = data_311[keep_columns2]
    data_311 = data_311.dropna()  
    data_311.columns = data_311.columns.str.strip().str.lower()  # Standardize column names
    
    return data_311
_311_datafile = 'data/311_data.json' 
df_311 = download_and_clean_311_data(_311_datafile)
df_311

,created_date,complaint_type,latitude,longitude,borough
0,2024-04-24T01:51:03.000,Noise - Residential,40.809497,-73.941700,MANHATTAN
1,2024-04-24T01:49:23.000,Noise - Residential,40.853718,-73.864825,BRONX
2,2024-04-24T01:48:53.000,Blocked Driveway,40.757165,-73.913862,QUEENS
3,2024-04-24T01:47:55.000,Noise - Residential,40.809497,-73.941700,MANHATTAN
4,2024-04-24T01:47:52.000,Blocked Driveway,40.603881,-73.960620,BROOKLYN
...,...,...,...,...,...
995,2024-04-23T21:46:22.000,UNSANITARY CONDITION,40.653202,-73.960533,BROOKLYN
996,2024-04-23T21:46:22.000,PAINT/PLASTER,40.653202,-73.960533,BROOKLYN
997,2024-04-23T21:46:18.000,Sidewalk Condition,40.761623,-73.960484,MANHATTAN
998,2024-04-23T21:45:34.000,New Tree Request,40.808476,-73.955124,MANHATTAN


In [90]:
def download_and_clean_tree_data(tree_datafile):
    # Download the tree data
    url_tree = "https://data.cityofnewyork.us/resource/5rq2-4hqu.geojson"
    headers = {"X-App-Token": NYC_DATA_APP_TOKEN}  
    response = requests.get(url_tree, headers=headers)
    response.raise_for_status()  

    filename_tree = data_dir / "treesdata.geojson"  
    with open(filename_tree, "w") as f:
        f.write(response.text)  

    data_tree = gpd.read_file("GeoJSON:data/treesdata.geojson") 
    data_tree = gpd.GeoDataFrame(data_tree, geometry='geometry')

    keep_columns3 = ["tree_id", 'created_at', 'block_id', 'status','spc_common','zip_city', "latitude", "longitude", 'geometry', 'health','zipcode','address']
    data_tree = data_tree[keep_columns3]  
    data_tree = data_tree.dropna(subset=["tree_id", "latitude", "longitude"])  
    data_tree.columns = data_tree.columns.str.strip().str.lower() 
    data_tree = data_tree.to_crs(epsg=2029) 

    return data_tree

tree_datafile = 'data/treesdata.geojson'
geodf_trees = download_and_clean_tree_data(tree_datafile)
#Convert geometry to WKT for SQL compatibility

geodf_trees

geodf_trees['geometry_wkt'] = geodf_trees['geometry'].apply(lambda geom: geom.wkt)
geodf_trees.drop(columns=['geometry'], inplace=True)


In [91]:
# Check the DataFrame to ensure correct structure and data types
print(geodf_trees.head())
print(geodf_trees.dtypes)


  tree_id  created_at block_id status       spc_common      zip_city  \
0  180683  08/27/2015   348711  Alive        red maple  Forest Hills   
1  200540  09/03/2015   315986  Alive          pin oak    Whitestone   
2  204026  09/05/2015   218365  Alive      honeylocust      Brooklyn   
3  204337  09/05/2015   217969  Alive      honeylocust      Brooklyn   
4  189565  08/30/2015   223043  Alive  American linden      Brooklyn   

      latitude     longitude health zipcode            address  \
0  40.72309177  -73.84421522   Fair   11375  108-005 70 AVENUE   
1  40.79411067  -73.81867946   Fair   11357   147-074 7 AVENUE   
2  40.71758074   -73.9366077   Good   11211  390 MORGAN AVENUE   
3  40.71353749  -73.93445616   Good   11211  1027 GRAND STREET   
4  40.66677776  -73.97597938   Good   11215       603 6 STREET   

                                   geometry_wkt  
0  POINT (1104574.2235162954 4532506.823261885)  
1  POINT (1106084.0750672938 4540576.364286132)  
2  POINT (1096811.87

In [92]:
data_dir = Path("data")

def load_and_clean_zillow_data(zillow_datafile):
    zillow_datafile = data_dir / "zillow_rent_data.csv"  
    zillow_df = pd.read_csv(zillow_datafile)
    keep_columns4 = ['RegionID','SizeRank','RegionName','StateName','CountyName','2024-01-31']
    zillow_df = zillow_df[keep_columns4]  
    zillow_df.columns = zillow_df.columns.str.strip().str.lower()  
    
    return zillow_df

zillow_datafile = 'data/zillow_rent_data.csv'  # Replace with your file path
cleaned_zillow = load_and_clean_zillow_data(zillow_datafile)
df_zillow = cleaned_zillow[cleaned_zillow['statename'] == 'NY']
df_zillow = df_zillow.rename(columns = {'regionname':'zipcode'})
df_zillow


,regionid,sizerank,zipcode,statename,countyname,2024-01-31
5,62093,7,11385,NY,Queens County,2880.094236
7,62019,9,11208,NY,Kings County,2704.611111
13,62046,16,11236,NY,Kings County,2487.500000
14,61807,17,10467,NY,Bronx County,2036.252778
15,62085,18,11373,NY,Queens County,2689.050000
...,...,...,...,...,...,...
6619,62283,24858,11932,NY,Suffolk County,52083.250000
6621,62281,25422,11930,NY,Suffolk County,66361.083333
6624,62306,27309,11959,NY,Suffolk County,55000.000000
6625,62309,27438,11962,NY,Suffolk County,125000.000000


In [71]:
def load_all_data():
    df_zipcodes = load_and_clean_zipcodes(zipcode_datafile)
    df_311 = download_and_clean_311_data(_311_datafile)
    geodf_trees = download_and_clean_tree_data(tree_datafile)
    df_zillow = load_and_clean_zillow_data(zillow_datafile)
    return (
        df_zipcodes,
        df_311,
        geodf_trees,
        df_zillow
    )

In [72]:
df_zipcodes, df_311, geodf_trees, df_zillow = load_all_data()

In [73]:
# Show basic info about each dataframe
df_zipcodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   zipcode     263 non-null    object 
 1   po_name     263 non-null    object 
 2   population  263 non-null    float64
 3   bldgzip     263 non-null    int64  
 4   area        263 non-null    float64
 5   county      263 non-null    object 
 6   cty_fips    263 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 14.5+ KB


In [93]:
# Show first 5 entries about each dataframe
df_zipcodes.head()

,zipcode,po_name,population,bldgzip,area,county,cty_fips
0,11436,Jamaica,18681.0,0,2.269930e+07,Queens,81
1,11213,Brooklyn,62426.0,0,2.963100e+07,Kings,47
2,11212,Brooklyn,83866.0,0,4.197210e+07,Kings,47
3,11225,Brooklyn,56527.0,0,2.369863e+07,Kings,47
4,11218,Brooklyn,72280.0,0,3.686880e+07,Kings,47


In [94]:
df_311.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 992 entries, 0 to 999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   created_date    992 non-null    object 
 1   complaint_type  992 non-null    object 
 2   latitude        992 non-null    float64
 3   longitude       992 non-null    float64
 4   borough         992 non-null    object 
dtypes: float64(2), object(3)
memory usage: 46.5+ KB


In [95]:
df_311.head()

,created_date,complaint_type,latitude,longitude,borough
0,2024-04-24T01:51:03.000,Noise - Residential,40.809497,-73.941700,MANHATTAN
1,2024-04-24T01:49:23.000,Noise - Residential,40.853718,-73.864825,BRONX
2,2024-04-24T01:48:53.000,Blocked Driveway,40.757165,-73.913862,QUEENS
3,2024-04-24T01:47:55.000,Noise - Residential,40.809497,-73.941700,MANHATTAN
4,2024-04-24T01:47:52.000,Blocked Driveway,40.603881,-73.960620,BROOKLYN


In [96]:
geodf_trees.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tree_id       1000 non-null   object
 1   created_at    1000 non-null   object
 2   block_id      1000 non-null   object
 3   status        1000 non-null   object
 4   spc_common    973 non-null    object
 5   zip_city      1000 non-null   object
 6   latitude      1000 non-null   object
 7   longitude     1000 non-null   object
 8   health        973 non-null    object
 9   zipcode       1000 non-null   object
 10  address       1000 non-null   object
 11  geometry_wkt  1000 non-null   object
dtypes: object(12)
memory usage: 93.9+ KB


In [97]:
geodf_trees.head()

,tree_id,created_at,block_id,status,spc_common,zip_city,latitude,longitude,health,zipcode,address,geometry_wkt
0,180683,08/27/2015,348711,Alive,red maple,Forest Hills,40.72309177,-73.84421522,Fair,11375,108-005 70 AVENUE,POINT (1104574.2235162954 4532506.823261885)
1,200540,09/03/2015,315986,Alive,pin oak,Whitestone,40.79411067,-73.81867946,Fair,11357,147-074 7 AVENUE,POINT (1106084.0750672938 4540576.364286132)
2,204026,09/05/2015,218365,Alive,honeylocust,Brooklyn,40.71758074,-73.9366077,Good,11211,390 MORGAN AVENUE,POINT (1096811.8748967955 4531258.709983518)
3,204337,09/05/2015,217969,Alive,honeylocust,Brooklyn,40.71353749,-73.93445616,Good,11211,1027 GRAND STREET,POINT (1097030.1308800587 4530824.103934255)
4,189565,08/30/2015,223043,Alive,American linden,Brooklyn,40.66677776,-73.97597938,Good,11215,603 6 STREET,POINT (1093936.2920769644 4525344.846361358)


In [98]:
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 310 entries, 5 to 6630
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   regionid    310 non-null    int64  
 1   sizerank    310 non-null    int64  
 2   zipcode     310 non-null    int64  
 3   statename   310 non-null    object 
 4   countyname  310 non-null    object 
 5   2024-01-31  310 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 17.0+ KB


In [99]:
df_zillow.head()

,regionid,sizerank,zipcode,statename,countyname,2024-01-31
5,62093,7,11385,NY,Queens County,2880.094236
7,62019,9,11208,NY,Kings County,2704.611111
13,62046,16,11236,NY,Kings County,2487.500000
14,61807,17,10467,NY,Bronx County,2036.252778
15,62085,18,11373,NY,Queens County,2689.050000


## Part 2: Storing Data

In [100]:
import psycopg2
from psycopg2 import sql

def setup_new_postgis_database(username, db_name):
    #Connect to the default database
    conn = psycopg2.connect(f"dbname=postgres user={username}")
    conn.autocommit = True 
    cursor = conn.cursor()
    
    #Create the new database
    try:
        cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Database {db_name} created successfully.")
    except psycopg2.DatabaseError as e:
        print(f"An error occurred: {e}")
        return
    
    #Connect to the newly created database
    conn.close()
    conn = psycopg2.connect(f"dbname={db_name} user={username}")
    conn.autocommit = True
    cursor = conn.cursor()
    
    #Enable PostGIS extension
    try:
        cursor.execute("CREATE EXTENSION postgis;")
        print("PostGIS extension created successfully.")
    except psycopg2.DatabaseError as e:
        print(f"An error occurred: {e}")
    
    #Close connection
    conn.close()

#Test
setup_new_postgis_database('alexander', 'aendta_db')


An error occurred: database "aendta_db" already exists



In [103]:
setup_new_postgis_database(DB_USER, DB_NAME)

An error occurred: database "aendta_db" already exists



### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [104]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [105]:
# If using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables.
# You may be creating more tables depending on how you're setting up your constraints/relationships
# or if you're completing the extra credit.
ZIPCODE_SCHEMA = """
CREATE TABLE IF NOT EXISTS zipcodes (
    zipcode VARCHAR(10) PRIMARY KEY,
    po_name VARCHAR(50),
    population DECIMAL(10, 2),
    bldgzip INTEGER,
    area FLOAT,
    county VARCHAR(50),
    cty_fips VARCHAR(10)
);

CREATE INDEX IF NOT EXISTS idx_zipcodes_po_name ON zipcodes (po_name);
CREATE INDEX IF NOT EXISTS idx_zipcodes_county ON zipcodes (county);
"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_311 (
    created_date TIMESTAMP,
    complaint_type VARCHAR(100),
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6),
    borough VARCHAR(50)
);

CREATE INDEX IF NOT EXISTS idx_nyc_311_complaint_type ON nyc_311 (complaint_type);
CREATE INDEX IF NOT EXISTS idx_nyc_311_borough ON nyc_311 (borough);
CREATE INDEX IF NOT EXISTS idx_nyc_311_geom ON nyc_311 USING gist (ST_SetSRID(ST_MakePoint(longitude, latitude), 4326));
"""

NYC_TREE_SCHEMA = """
CREATE TABLE IF NOT EXISTS nyc_trees (
    tree_id BIGINT PRIMARY KEY,
    created_at DATE,
    block_id BIGINT,
    status VARCHAR(20),
    spc_common VARCHAR(50),
    zip_city VARCHAR(50),
    latitude DECIMAL(9, 6),
    longitude DECIMAL(9, 6),
    geometry_wkt VARCHAR(255), 
    health VARCHAR(20),
    zipcode VARCHAR(10),
    address VARCHAR(100)
);

CREATE INDEX IF NOT EXISTS idx_nyc_trees_spc_common ON nyc_trees (spc_common);
CREATE INDEX IF NOT EXISTS idx_nyc_trees_zipcode ON nyc_trees (zipcode);
CREATE INDEX IF NOT EXISTS idx_nyc_trees_geom ON nyc_trees USING gist (geometry);
"""


ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS zillow_rents (
    regionid BIGINT PRIMARY KEY,
    sizerank BIGINT,
    zipcode VARCHAR(10),
    statename VARCHAR(50),
    countyname VARCHAR(50),
    rental_date DATE,
    avg_rent DECIMAL(10, 2)
);
"""


In [106]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [107]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [109]:
def write_dataframes_to_table(tablename_to_dataframe, db_url):
    if not isinstance(tablename_to_dataframe, dict):
        raise ValueError("Expected a dictionary mapping table names to DataFrames.")

    #Write each DataFrame to the specified table
    for tablename, dataframe in tablename_to_dataframe.items():
        if dataframe is None or dataframe.empty:
            raise ValueError(f"The DataFrame for table '{tablename}' is empty or not provided.")

        try:
            dataframe.to_sql(tablename, engine, if_exists='replace', index=False)
            print(f"Successfully wrote data to table '{tablename}'")
        except Exception as e:
            print(f"Error writing data to table '{tablename}': {e}")
            raise

In [110]:
tablename_to_dataframe = {
    "zipcodes": df_zipcodes,
    "complaints": df_311,
    "trees": geodf_trees,
    "rents": df_zillow,
}

In [111]:
write_dataframes_to_table(tablename_to_dataframe, DB_URL)

Successfully wrote data to table 'zipcodes'
Successfully wrote data to table 'complaints'
Successfully wrote data to table 'trees'
Successfully wrote data to table 'rents'


#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)